In [2]:
###################
## AUTHORIZATION ##
###################
import tekore as tk #Spotify Web API
def authorize():
    CLIENT_ID = "31b908bc909041a5a2898268aaf89962"
    CLIENT_SECRET = "41a1563f82c44bffaac02437b11e8fc3"
    app_token = tk.request_client_token(CLIENT_ID, CLIENT_SECRET)
    return tk.Spotify(app_token)

In [3]:
#################
## PREPARATION ##
#################

import authorization
import pandas as pd
from tqdm import tqdm
import time

# Authorize and call access object "sp"
sp = authorization.authorize()

# Get all genres
genres = sp.recommendation_genre_seeds()

# Set number of recommendations per genre
n_recs = 10

# Initiate a dictionary with all the information you want to crawl
data_dict = {"id":[], "genre":[], "track_name":[], "artist_name":[],
             "valence":[], "energy":[]}

################
## CRAWL DATA ##
################

# Get recs for every genre
for g in tqdm(genres):
    
    # Get n recommendations
    recs = sp.recommendations(genres = [g], limit = n_recs)
    # json-like string to dict
    recs = eval(recs.json().replace("null", "-999").replace("false", "False").replace("true", "True"))["tracks"]
    
    # Crawl data from each track
    for track in recs:
        # ID and Genre
        data_dict["id"].append(track["id"])
        data_dict["genre"].append(g)
        # Metadata
        track_meta = sp.track(track["id"])
        data_dict["track_name"].append(track_meta.name)
        data_dict["artist_name"].append(track_meta.album.artists[0].name)
        # Valence and energy
        track_features = sp.track_audio_features(track["id"])
        data_dict["valence"].append(track_features.valence)
        data_dict["energy"].append(track_features.energy)
        
        # Wait 0.2 seconds per track so that the api doesnt overheat
        time.sleep(0.2)
        
##################
## PROCESS DATA ##
##################

# Store data in dataframe
df = pd.DataFrame(data_dict)

# Drop duplicates
df.drop_duplicates(subset = "id", keep = "first", inplace = True)
df.to_csv("valence_arousal_dataset.csv", index = False)

100%|██████████| 126/126 [13:13<00:00,  6.30s/it]


In [4]:
import numpy as np
from numpy.linalg import norm
import random
import pandas as pd
########################
## recommend function ##
########################
df=pd.read_csv("valence_arousal_dataset.csv")

df["mood_vec"] = df[["valence", "energy"]].values.tolist()
def recommend(valence,energy, ref_df):
    
    featurevector = np.array([valence, energy])
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(featurevector-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
    
    # Return n recommendations
    return ref_df_sorted.iloc[0]

In [5]:
def get_n_recs(point1,point2,n_recs):
    x=recommend(point1[0],point1[1],df)
    print(x)
    tracks=[{"name":x["track_name"],"id":x["id"]}]
    n=n_recs-1
    x_change=(point2[0]-point1[0])/n
    y_change=(point2[1]-point1[1])/n
    x_val=point1[0]
    y_val=point1[1]
    for i in range(n):
        x_val+=x_change
        y_val+=y_change
        x=recommend(x_val,y_val,df)
        print(x)
        tracks.append({"name":x["track_name"],"id":x["id"]})
    return tracks

In [6]:
print(get_n_recs(point1=[0.0,0.0],point2=[1.0,1.0],n_recs=5))


id                                        0Vjz7AGR5eDAa1CoSUBIZL
genre                                                  classical
track_name     Clarinet Concertino: Concertino for Clarinet a...
artist_name                                        Nicola LeFanu
valence                                                    0.039
energy                                                   0.00702
mood_vec                                        [0.039, 0.00702]
distances                                               0.039627
Name: 174, dtype: object
id             1SxNHQSrK1U9d4gZ866LBX
genre                             mpb
track_name                    Segunda
artist_name                 Gal Costa
valence                         0.249
energy                          0.234
mood_vec               [0.249, 0.234]
distances                    0.016031
Name: 780, dtype: object
id             4Odr8ViOk7wZrPlJyIL15X
genre                      show-tunes
track_name                 Suppertime
artist_name 

: 